In [1]:
import pandas as pd
import networkx as nx
import re

# Load datasets
fuel_rates = pd.read_excel('Quarterly Fuel rates.xlsx').dropna().drop_duplicates()
admin_mail = pd.read_excel('GeneralAdmin_Mail.xlsx').dropna().drop_duplicates()
merged_data = pd.read_excel('merged_excel_file.xlsx').dropna().drop_duplicates()
cleaned_dataset = pd.read_csv("/content/cleaned_dataset (2).csv").dropna().drop_duplicates()

# Standardize column names
def standardize_columns(df):
    df.columns = [col.strip().lower() for col in df.columns]
    return df

fuel_rates = standardize_columns(fuel_rates)
admin_mail = standardize_columns(admin_mail)
merged_data = standardize_columns(merged_data)
cleaned_dataset = standardize_columns(cleaned_dataset)

# Normalize city names
def normalize_city_name(city_name):
    return re.sub(r'[^a-z0-9]+', '', city_name.lower().strip())

# Prepare route data
route_data = cleaned_dataset[['city1', 'city2', 'nsmiles', 'passengers', 'fare']].copy()
route_data['city1'] = route_data['city1'].apply(normalize_city_name)
route_data['city2'] = route_data['city2'].apply(normalize_city_name)

# Initialize graph
G = nx.DiGraph()
average_flight_speed = 500  # mph

# Add edges to the graph
for _, row in route_data.iterrows():
    origin = row['city1']
    destination = row['city2']
    distance = row['nsmiles']
    passengers = row['passengers']
    fare = row['fare']

    travel_time = distance / average_flight_speed
    airport_charges = 50 + 0.1 * distance
    revenue = fare * passengers
    profit = revenue - airport_charges

    G.add_edge(origin, destination, weight=travel_time, distance=distance,
               passengers=passengers, fare=fare, charges=airport_charges, profit=profit)

# Display available cities
print("Graph Nodes (normalized city names):\n", list(G.nodes))

# User input for cities
start_city = normalize_city_name(input("Enter the start city: ").strip())
end_city = normalize_city_name(input("Enter the end city: ").strip())

# Function to find connecting flights
def find_all_routes(graph, start, end, max_paths=3):
    try:
        all_paths = list(nx.shortest_simple_paths(graph, source=start, target=end, weight='weight'))
        best_paths = []
        for path in all_paths[:max_paths]:
            travel_time = sum(graph[path[i]][path[i + 1]]['weight'] for i in range(len(path) - 1))
            profit = sum(graph[path[i]][path[i + 1]]['profit'] for i in range(len(path) - 1))
            charges = sum(graph[path[i]][path[i + 1]]['charges'] for i in range(len(path) - 1))
            layovers = path[1:-1] if len(path) > 2 else []
            best_paths.append({
                'path': path,
                'travel_time': travel_time,
                'profit': profit,
                'charges': charges,
                'layovers': layovers
            })
        return best_paths
    except nx.NetworkXNoPath:
        return []

# Validate cities and find routes
if start_city in G.nodes and end_city in G.nodes:
    # Check direct flight
    if G.has_edge(start_city, end_city):
        direct_distance = G[start_city][end_city]['distance']
        direct_time = G[start_city][end_city]['weight']
        direct_profit = G[start_city][end_city]['profit']
        direct_charges = G[start_city][end_city]['charges']

        print("\nDirect Flight Available:")
        print(f"  Path: {start_city} → {end_city}")
        print(f"  Distance: {direct_distance} miles")
        print(f"  Total Travel Time: {direct_time:.2f} hours")
        print(f"  Profit: ${direct_profit:.2f}")
        print(f"  Airport Charges: ${direct_charges:.2f}")
    else:
        print("\nNo direct flight available.")

    # Find connecting flights
    connecting_routes = find_all_routes(G, start_city, end_city)
    if connecting_routes:
        print("\nConnecting Flight Options:")
        for idx, route in enumerate(connecting_routes, start=1):
            print(f"\nRoute {idx}:")
            print(f"  Path: {' → '.join(route['path'])}")
            print(f"  Layovers: {', '.join(route['layovers']) if route['layovers'] else 'None'}")
            print(f"  Total Travel Time: {route['travel_time']:.2f} hours")
            print(f"  Profit: ${route['profit']:.2f}")
            print(f"  Total Airport Charges: ${route['charges']:.2f}")
    else:
        print("\nNo connecting flight options available.")
else:
    print("Invalid city names. Please check the available cities and try again.")


FileNotFoundError: [Errno 2] No such file or directory: 'Quarterly Fuel rates.xlsx'

In [ ]:
import pandas as pd
import networkx as nx
import re

# Load and clean datasets
def load_and_clean_data():
    fuel_rates = pd.read_excel('Quarterly Fuel rates.xlsx').dropna().drop_duplicates()
    admin_mail = pd.read_excel('GeneralAdmin_Mail.xlsx').dropna().drop_duplicates()
    merged_data = pd.read_excel('merged_excel_file.xlsx').dropna().drop_duplicates()
    cleaned_dataset = pd.read_csv("/content/cleaned_dataset (2).csv").dropna().drop_duplicates()
    return fuel_rates, admin_mail, merged_data, cleaned_dataset

# Standardize column names
def standardize_columns(df):
    df.columns = [col.strip().lower() for col in df.columns]
    return df

# Normalize city names
def normalize_city_name(city_name):
    return re.sub(r'[^a-z0-9]+', '', city_name.lower().strip())

# Prepare route data
def prepare_route_data(cleaned_dataset):
    route_data = cleaned_dataset[['city1', 'city2', 'nsmiles', 'passengers', 'fare']].copy()
    route_data['city1'] = route_data['city1'].apply(normalize_city_name)
    route_data['city2'] = route_data['city2'].apply(normalize_city_name)
    return route_data

# Build the graph
def build_graph(route_data):
    G = nx.DiGraph()
    average_flight_speed = 500  # mph

    for _, row in route_data.iterrows():
        origin = row['city1']
        destination = row['city2']
        distance = row['nsmiles']
        passengers = row['passengers']
        fare = row['fare']

        travel_time = distance / average_flight_speed
        airport_charges = 50 + 0.1 * distance
        revenue = fare * passengers
        profit = revenue - airport_charges

        G.add_edge(origin, destination, weight=travel_time, distance=distance,
                   passengers=passengers, fare=fare, charges=airport_charges, profit=profit)
    return G

# Define regional hubs
def define_regional_hubs():
    east_coast_cities = ['new york city, ny', 'boston, ma', 'washington, dc', 'philadelphia, pa', 'charlotte, nc']
    west_coast_cities = ['los angeles, ca', 'san francisco, ca', 'seattle, wa', 'portland, or', 'san diego, ca']
    sunbelt_cities = ['dallas/fort worth, tx', 'phoenix, az', 'atlanta, ga', 'houston, tx', 'austin, tx']
    northern_cities = ['chicago, il', 'minneapolis, mn', 'detroit, mi', 'cleveland, oh']

    return {
        'East Coast': [normalize_city_name(city) for city in east_coast_cities],
        'West Coast': [normalize_city_name(city) for city in west_coast_cities],
        'Sunbelt': [normalize_city_name(city) for city in sunbelt_cities],
        'Northern': [normalize_city_name(city) for city in northern_cities]
    }

# Find best layover path
def find_best_layover_path(graph, start, end, layover_hubs):
    best_path = None
    best_time = float('inf')
    best_profit = float('-inf')

    for hub in layover_hubs:
        if hub in graph.nodes:
            try:
                path_to_hub = nx.shortest_path(graph, source=start, target=hub, weight='weight')
                path_from_hub = nx.shortest_path(graph, source=hub, target=end, weight='weight')
                full_path = path_to_hub + path_from_hub[1:]
                travel_time = sum(graph[u][v]['weight'] for u, v in zip(full_path[:-1], full_path[1:]))
                total_profit = sum(graph[u][v]['profit'] for u, v in zip(full_path[:-1], full_path[1:]))

                if travel_time < best_time or (travel_time == best_time and total_profit > best_profit):
                    best_path = full_path
                    best_time = travel_time
                    best_profit = total_profit
            except nx.NetworkXNoPath:
                continue

    return best_path, best_time, best_profit

# Main function
def main():
    # Load data
    _, _, _, cleaned_dataset = load_and_clean_data()

    # Prepare data
    route_data = prepare_route_data(cleaned_dataset)
    G = build_graph(route_data)

    # Define hubs
    regional_hubs = define_regional_hubs()

    # Display available cities
    print("Graph Nodes (normalized city names):\n", list(G.nodes))

    # User input for cities
    start_city = normalize_city_name(input("Enter the start city: ").strip())
    end_city = normalize_city_name(input("Enter the end city: ").strip())

    # Validate cities and find routes
    if start_city in G.nodes and end_city in G.nodes:
        # Check direct flight
        if G.has_edge(start_city, end_city):
            direct_distance = G[start_city][end_city]['distance']
            direct_time = G[start_city][end_city]['weight']
            direct_profit = G[start_city][end_city]['profit']
            direct_charges = G[start_city][end_city]['charges']

            print("\nDirect Flight Available:")
            print(f"  Path: {start_city} → {end_city}")
            print(f"  Distance: {direct_distance} miles")
            print(f"  Total Travel Time: {direct_time:.2f} hours")
            print(f"  Profit: ${direct_profit:.2f}")
            print(f"  Airport Charges: ${direct_charges:.2f}")
        else:
            # Find best routes through various regions
            print("\nConnecting Flight Options:")
            for region_name, hubs in regional_hubs.items():
                path, travel_time, profit = find_best_layover_path(G, start_city, end_city, hubs)
                if path:
                    layovers = path[1:-1]
                    print(f"\n{region_name} Route:")
                    print(f"  Path: {' → '.join(path)}")
                    print(f"  Layovers: {', '.join(layovers) if layovers else 'None'}")
                    print(f"  Total Travel Time: {travel_time:.2f} hours")
                    print(f"  Profit: ${profit:.2f}")
    else:
        print("Invalid city names. Please check the available cities and try again.")

# Run the main function
if __name__ == "__main__":
    main()


Graph Nodes (normalized city names):
 ['allentownbethlehemeastonpa', 'tampaflmetropolitanarea', 'albuquerquenm', 'dallasfortworthtx', 'phoenixaz', 'bostonmametropolitanarea', 'washingtondcmetropolitanarea', 'chicagoil', 'houstontx', 'newyorkcitynymetropolitanarea', 'sanfranciscocametropolitanarea', 'losangelescametropolitanarea', 'nantucketma', 'coloradospringsco', 'pittsburghpa', 'denverco', 'tucsonaz', 'hartfordct', 'seattlewa', 'elpasotx', 'nashvilletn', 'stlouismo', 'myrtlebeachsc', 'jacksonvillefl', 'detroitmi', 'desmoinesia', 'orlandofl', 'panamacityfl', 'valparaisofl', 'fresnoca', 'minneapolisstpaulmn', 'fortmyersfl', 'greenvillespartanburgsc', 'spokanewa', 'grandrapidsmi', 'harlingensanbenitotx', 'lasvegasnv', 'indianapolisin', 'miamiflmetropolitanarea', 'littlerockar', 'lubbocktx', 'louisvilleky', 'midlandodessatx', 'sacramentoca', 'kansascitymo', 'sanantoniotx', 'memphistn', 'omahane', 'milwaukeewi', 'neworleansla', 'sandiegoca', 'norfolkvametropolitanarea', 'pensacolafl', 'w

In [ ]:
import pandas as pd
import networkx as nx
import re

# Load datasets
fuel_rates = pd.read_excel('Quarterly Fuel rates.xlsx').dropna().drop_duplicates()
admin_mail = pd.read_excel('GeneralAdmin_Mail.xlsx').dropna().drop_duplicates()
merged_data = pd.read_excel('merged_excel_file.xlsx').dropna().drop_duplicates()
cleaned_dataset = pd.read_csv("/content/cleaned_dataset (2).csv").dropna().drop_duplicates()

# Standardize column names
def standardize_columns(df):
    df.columns = [col.strip().lower() for col in df.columns]
    return df

fuel_rates = standardize_columns(fuel_rates)
admin_mail = standardize_columns(admin_mail)
merged_data = standardize_columns(merged_data)
cleaned_dataset = standardize_columns(cleaned_dataset)

# Normalize city names
def normalize_city_name(city_name):
    return re.sub(r'[^a-z0-9]+', '', city_name.lower().strip())

# Prepare route data
route_data = cleaned_dataset[['city1', 'city2', 'nsmiles', 'passengers', 'fare']].copy()
route_data['city1'] = route_data['city1'].apply(normalize_city_name)
route_data['city2'] = route_data['city2'].apply(normalize_city_name)

# Initialize graph
G = nx.DiGraph()
average_flight_speed = 500  # mph

# Add edges to the graph
for _, row in route_data.iterrows():
    origin = row['city1']
    destination = row['city2']
    distance = row['nsmiles']
    passengers = row['passengers']
    fare = row['fare']

    travel_time = distance / average_flight_speed
    airport_charges = 50 + 0.1 * distance
    revenue = fare * passengers
    profit = revenue - airport_charges

    G.add_edge(origin, destination, weight=travel_time, distance=distance,
               passengers=passengers, fare=fare, charges=airport_charges, profit=profit)

# Display available cities
print("Graph Nodes (normalized city names):\n", list(G.nodes))

# User input for cities
start_city = normalize_city_name(input("Enter the start city: ").strip())
end_city = normalize_city_name(input("Enter the end city: ").strip())

# Function to find connecting flights
def find_all_routes(graph, start, end, max_paths=3):
    try:
        all_paths = list(nx.shortest_simple_paths(graph, source=start, target=end, weight='weight'))
        best_paths = []
        for path in all_paths[:max_paths]:
            travel_time = sum(graph[path[i]][path[i + 1]]['weight'] for i in range(len(path) - 1))
            profit = sum(graph[path[i]][path[i + 1]]['profit'] for i in range(len(path) - 1))
            charges = sum(graph[path[i]][path[i + 1]]['charges'] for i in range(len(path) - 1))
            fare = sum(graph[path[i]][path[i + 1]]['fare'] for i in range(len(path) - 1))
            layovers = path[1:-1] if len(path) > 2 else []
            best_paths.append({
                'path': path,
                'travel_time': travel_time,
                'profit': profit,
                'charges': charges,
                'layovers': layovers,
                'fare': fare
            })
        return best_paths
    except nx.NetworkXNoPath:
        return []

# Validate cities and find routes
if start_city in G.nodes and end_city in G.nodes:
    # Check direct flight
    if G.has_edge(start_city, end_city):
        direct_distance = G[start_city][end_city]['distance']
        direct_time = G[start_city][end_city]['weight']
        direct_profit = G[start_city][end_city]['profit']
        direct_charges = G[start_city][end_city]['charges']
        direct_fare = G[start_city][end_city]['fare']

        print("\nDirect Flight Available:")
        print(f"  Path: {start_city} → {end_city}")
        print(f"  Distance: {direct_distance} miles")
        print(f"  Total Travel Time: {direct_time:.2f} hours")
        print(f"  Profit: ${direct_profit:.2f}")
        print(f"  Airport Charges: ${direct_charges:.2f}")
        print(f"  Cost per Passenger - Second Class: ${direct_fare:.2f}")
        print(f"  Cost per Passenger - First Class: ${direct_fare * 1.75:.2f}")  # Assuming 1.75x economy fare
    else:
        print("\nNo direct flight available.")

    # Find connecting flights
    connecting_routes = find_all_routes(G, start_city, end_city)
    if connecting_routes:
        print("\nConnecting Flight Options:")
        for idx, route in enumerate(connecting_routes, start=1):
            print(f"\nRoute {idx}:")
            print(f"  Path: {' → '.join(route['path'])}")
            print(f"  Layovers: {', '.join(route['layovers']) if route['layovers'] else 'None'}")
            print(f"  Total Travel Time: {route['travel_time']:.2f} hours")
            print(f"  Profit: ${route['profit']:.2f}")
            print(f"  Total Airport Charges: ${route['charges']:.2f}")
            print(f"  Cost per Passenger - Second Class: ${route['fare']:.2f}")
            print(f"  Cost per Passenger - First Class: ${route['fare'] * 1.75:.2f}")  # Assuming 1.75x economy fare
    else:
        print("\nNo connecting flight options available.")
else:
    print("Invalid city names. Please check the available cities and try again.")


Graph Nodes (normalized city names):
 ['allentownbethlehemeastonpa', 'tampaflmetropolitanarea', 'albuquerquenm', 'dallasfortworthtx', 'phoenixaz', 'bostonmametropolitanarea', 'washingtondcmetropolitanarea', 'chicagoil', 'houstontx', 'newyorkcitynymetropolitanarea', 'sanfranciscocametropolitanarea', 'losangelescametropolitanarea', 'nantucketma', 'coloradospringsco', 'pittsburghpa', 'denverco', 'tucsonaz', 'hartfordct', 'seattlewa', 'elpasotx', 'nashvilletn', 'stlouismo', 'myrtlebeachsc', 'jacksonvillefl', 'detroitmi', 'desmoinesia', 'orlandofl', 'panamacityfl', 'valparaisofl', 'fresnoca', 'minneapolisstpaulmn', 'fortmyersfl', 'greenvillespartanburgsc', 'spokanewa', 'grandrapidsmi', 'harlingensanbenitotx', 'lasvegasnv', 'indianapolisin', 'miamiflmetropolitanarea', 'littlerockar', 'lubbocktx', 'louisvilleky', 'midlandodessatx', 'sacramentoca', 'kansascitymo', 'sanantoniotx', 'memphistn', 'omahane', 'milwaukeewi', 'neworleansla', 'sandiegoca', 'norfolkvametropolitanarea', 'pensacolafl', 'w